<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/Transportation%20Problem%20(with%20emission%20reduction%20strategies).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transportation Problem with emission reduction strategies

## Required packages

In [74]:
# Load required pyhton packages
!pip install pulp
import pulp
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Data

In [75]:
# Supply node
S = ['Hamburg', 'Berlin', 'Munich']

# Supply
s = {'Hamburg': 100, 'Berlin': 200, 'Munich': 70}

# Demand nodes
D = ['Bremen', 'Cologne', 'Dresden']

# Demand
d = {'Bremen': 80, 'Cologne': 40, 'Dresden': 90}

# Transport costs
c = {
    'Hamburg': {'Bremen': 4, 'Cologne': 3, 'Dresden': 8},
    'Berlin': {'Bremen': 6, 'Cologne': 8, 'Dresden': 9},
    'Munich': {'Bremen': 7, 'Cologne': 6, 'Dresden': 5}
}


## PuLP Model

In [76]:
def model_all_inclusive(S, s, D, d, c, e,
                        carbon_tax=None,
                        carbon_limit=None,
                        p_buy=None, p_sell=None, initial_amount=None,
                        obj=None):
  # Model
  model = pulp.LpProblem(name='Transport_Problem_Min_Emissions',
                        sense=pulp.constants.LpMinimize)

  # Decision variables
  x = pulp.LpVariable.dicts(name='x', indices=(S, D), lowBound=0, cat='Integer')

  # Dummy decision variables for reporting
  costs = pulp.LpVariable('costs', lowBound=0, upBound=None, cat=pulp.LpContinuous)
  emissions = pulp.LpVariable('emissions', lowBound=0, upBound=None, cat=pulp.LpContinuous)

  # Decision variables fro carbon trading scenario
  if initial_amount is not None:
    buy = pulp.LpVariable('buy', lowBound=0, upBound=None, cat=pulp.LpContinuous)
    sell = pulp.LpVariable('sell', lowBound=0, upBound=None, cat=pulp.LpContinuous)

  # (1) Objective
  if obj == 'emissions':
    model += emissions, '(1) emissions'
  else:
    if carbon_tax is not None:
      model += costs + emissions*carbon_tax, '(1) carbon tax'
    elif initial_amount is not None:
      model += costs + p_buy * buy - p_sell * sell, '(1) carbon trading'
    else:
      model += costs, '(1) costs'

  # add dummy decision variables to the model
  model += costs == pulp.lpSum(c[i][j] * x[i][j] for i in S for j in D)
  model += emissions == pulp.lpSum(e[i][j] * x[i][j] for i in S for j in D)

  # (2)
  for i in S:
    model += s[i] >= pulp.lpSum(x[i][j] for j in D), '(2)_' + str(i)

  # (2)
  for j in D:
    model += d[j] == pulp.lpSum(x[i][j] for i in S), '(3)_' + str(j)

  # carbon limit
  if carbon_limit is not None:
    model += carbon_limit >= pulp.lpSum(x[i][j]*e[i][j] for i in S for j in D), 'Carbon_Limit'

  # carbon trading
  if initial_amount is not None:
    model += emissions <= initial_amount + buy - sell, 'Carbon_Trade'

  return model

## Solving

In [77]:
def solve_and_print(model, extended=True):
  model.solve()

  if extended:
    # get status
    print("Status:", pulp.LpStatus[model.status])

    # get objective value
    print('Objective value:', round(pulp.value(model.objective), 2))

    print("All variables:")
    for v in model.variables():
      if v.varValue > 0:
        print(v.name, "=", v.varValue)
  else:
    for v in model.variables():
      if v.varValue > 0 and v.name in ['costs', 'emissions']:
        print(v.name, "=", v.varValue)

## Experiments

In [78]:
print('min costs')
model = model_all_inclusive(S=S, s=s, D=D, d=d, c=c, e=e, obj='costs')
solve_and_print(model, extended=False)

print('')
print('min emissions')
model = model_all_inclusive(S=S, s=s, D=D, d=d, c=c, e=e, obj='emissions')
solve_and_print(model, extended=False)

# for carbon_tax in np.arange(0.1, 1.6, 0.2):
#   print('')
#   print('min costs with carbon tax ' + str(round(carbon_tax, 1)))
#   model = model_all_inclusive(S=S, s=s, D=D, d=d, c=c, e=e, obj='costs',
#                               carbon_tax=round(carbon_tax, 1))
#   solve_and_print(model, extended=False)


# for carbon_limit in np.arange(1000, 1100, 50):
#   print('')
#   print('min costs with carbon limit ' + str(round(carbon_limit, 1)))
#   model = model_all_inclusive(S=S, s=s, D=D, d=d, c=c, e=e, obj='costs',
#                               carbon_limit=float(carbon_limit))
#   solve_and_print(model, extended=False)

print('')
print('min costs with carbon trading' )
model = model_all_inclusive(S=S, s=s, D=D, d=d, c=c, e=e, obj='costs',
                            p_buy=1, p_sell=1, initial_amount=1200)
solve_and_print(model, extended=False)


min costs
costs = 1010.0
emissions = 1650.0

min emissions
costs = 1470.0
emissions = 1080.0

min costs with carbon trading
costs = 1450.0
emissions = 1100.0
